# Install

In [1]:
# !pip install transformers
# !pip install langchain
# !pip install python-dotenv
# !pip install pypdf
# !pip install chromadb
# !pip install sentence-transformers
# !pip install openai
# !pip install -qU langchain-openai

# API token * key

In [2]:
import os
from dotenv import load_dotenv

os.environ['HUGGINGFACEHUB_API_TOKEN'] 

# API KEY 정보로드
load_dotenv()
# print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

True

# Import

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_community.llms import HuggingFaceEndpoint
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.embeddings import (
    HuggingFaceEmbeddings,
    HuggingFaceBgeEmbeddings,
)
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import TextLoader
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# Data preprocessing

* pdf에서 텍스트 추출
* 취소선과 겹치는 문장의 경우 < del>  <> 이런식으로 처리
* '. .' 과 같은 불필요한 단어 삭제

In [ ]:
import fitz
import re

def find_text_by_red_strikethrough_status(pdf_path):
    document = fitz.open(pdf_path)
    strikethrough_texts = []
    non_strikethrough_texts = []
    full_texts = []

    for page_number in range(len(document)):
        page = document[page_number]
        words = page.get_text("words")  # 단어와 그 위치를 반환
        paths = page.get_drawings()  # 페이지의 그래픽 요소를 추출

        strikethrough_lines = []

        # 그림 요소 중에서 선과 사각형을 검사하여 빨간색 취소선으로 판단
        for path in paths:
            color = path["color"]
            # 선의 색상이 빨간색인 경우에만 처리
            if color == (1, 0, 0):  # RGB 색상으로 빨간색 확인
                for item in path["items"]:
                    if item[0] == "l":  # 선인 경우
                        p1, p2 = item[1:]
                        if p1.y == p2.y:  # 수평선이면
                            rect = fitz.Rect(p1.x, p1.y - 1, p2.x, p2.y + 1)
                            strikethrough_lines.append(rect)
                    elif item[0] == "re":  # 사각형인 경우
                        rect = item[1]
                        if rect.width > rect.height and rect.height < 3:  # 넓이가 높이보다 많이 크고 높이가 3pt 이하이면
                            strikethrough_lines.append(rect)

        # 각 단어와 취소선이 겹치는지 검사
        same_line = words[0][5]
        previous_strike = False
        strike_line = ''
        line = ''
        for word in words:
            word_rect = fitz.Rect(word[:4])  # 단어의 위치
            strikethrough_found = False
            for line_rect in strikethrough_lines:
                if word_rect.intersects(line_rect):  # 겹치면
                    strikethrough_found = True
                    break
            if not strikethrough_found:  # 취소선이 없으면
                non_strikethrough_texts.append(word[4:6])  # 취소선이 적용되지 않은 단어 추가
                if same_line != word[5]:
                    same_line = word[5]
                    line += '\n'

                line = line + ' ' + word[4]
                
                if strikethrough_found != previous_strike:
                    full_texts.append('<del>' + strike_line + '<>')
                    strike_line = ''
                previous_strike = False
            else:
                strikethrough_texts.append(word[4:6])  # 취소선이 적용된 단어 추가
                strike_line = strike_line  + ' ' + word[4]
                if strikethrough_found != previous_strike:
                    full_texts.append(line + '\n')
                    line=''
                previous_strike = True
        full_texts.append(line)
    document.close()
    return strikethrough_texts, non_strikethrough_texts, full_texts

def re_text(full_texts):
    full = ''
    for text in full_texts:
        cleaned_text = re.sub(r'\.\s*\.', '', text)
        full += cleaned_text
    return full

pdf_path = ['real_data_ex.pdf', 'real_data_ex2.pdf']
docs = []
for pdf in pdf_path:
    strikethrough_texts, non_strikethrough_texts, full_texts = find_text_by_red_strikethrough_status(pdf)
    texts = re_text(full_texts)
    docs.append((pdf, texts))
docs

# Make chunk data - vector db에 저장하기전 데이터 만들기

* 앞서 pdf에서 추출한 데이터 활용
* vector DB에 저장하기 위해 pdf를 chunk 단위로 잘라 저장할 필요가 있음
* RecursiveCharacterTextSplitter를 이용하여 2000개씩 자르고, 200개씩 겹쳐서 문서가 연결되게끔 split
    * ex) "나는 ai 파트를 맡고", "ai 파트를 맡고 있어요" -> 이런식으로 겹쳐서 저장

* 메타데이터로 pdf 문서 이름을 같이 저장
* create_document를 이용해서 2000개로 자른 텍스트와 메타데이터 저장
    * Document(page_content='< del>  <> 1 주의 금액 400원  , metadata={'source': 'real_data_ex.pdf'}) 형식

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 200,
    length_function = len,
)

data = []
for pdf, texts in docs:
    split_texts = text_splitter.split_text(texts)
    metadata = [{'source': pdf}] * len(split_texts)
    pages = text_splitter.create_documents(split_texts, metadatas=metadata)
    data.append(pages)
data

# Store data to vector DB
* vector db에 저장하기 위한 임베딩은 openai의 임베딩을 사용
* 앞서 Document(page_content, metadata) 형식의 데이터를 db에 저장
* persist_directory는 로컬에 저장할 폴더 이름
* vector_index._collection.count()로 현재 DB에 몇 개의 Document가 저장되어 있는지 알 수 있다. 

In [ ]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_openai import OpenAIEmbeddings

# directory = 'chroma_store_hugging'
directory = 'chroma_store_open'

embeddings = HuggingFaceEmbeddings()
# embeddings_model = HuggingFaceEmbeddings(
#     model_name='jhgan/ko-sbert-nli',
#     model_kwargs={'device':'cpu'},
#     encode_kwargs={'normalize_embeddings':True},
# )

embeddings_open = OpenAIEmbeddings(model="text-embedding-3-large")

for pages in data:
    vector_index = Chroma.from_documents(
        pages, # Documents
        embedding = embeddings_open, # Text embedding model
        persist_directory=directory # persists the vectors to the file system
        )
# vector_index.persist()
print('count: ', vector_index._collection.count())

# Delete all data
* db에 있는 정보 삭제

In [ ]:
# # # 정보 삭제
# ids = vector_index.get(0)['ids']

# print('before: ', vector_index._collection.count())
# for i in ids:
#     vector_index._collection.delete(ids=i)
# print('after :', vector_index._collection.count())

# Load
* db에 따로 저장하지 않고 저장되어 있는 것만 사용할 경우 load 가능

In [ ]:
embeddings_open = OpenAIEmbeddings(model="text-embedding-3-large")
vector_index = Chroma(persist_directory='chroma_store_open', embedding_function=embeddings_open)
print('count: ', vector_index._collection.count())

count:  11


# Huggingface model (prompt)
* huggingface model(llm)을 사용
    * prompt는 영어로 작성된 prompt 사용 - prompt에 따라 성능이 달라짐
    * retriever는 벡터 DB에서 내가 질문한 것과 유사도가 높은 Document를 k개 고른다.
    * ConversationalRetrievalChain으로 llm, retriever, prompt를 사용
    * 물어보는 질문에 대해 db에서 유사한 Document를 찾고 그것을 기반으로 답변 생성
    * chat_history에 답변 저장
    

In [ ]:
chat_history = []

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

prompt = ChatPromptTemplate.from_template("""
    you must not use information in the form of <del><> except when requested
    The metadata being different means the documents are different. 
    I want to obtain information seperately for each document.
    you answer me only using context for question:
    <context>
    {context}
    </context>
    
    Question: {question}
    i want korean answer and don't print english""")

prompt_korean = ChatPromptTemplate.from_template("""
    너는 문서를 보고 대답을 하는 전문가야 모르는 답은 모른다고 답을 해줘
    문서를 보면 "말소기록()" 괄호로 묶인 텍스트가 있는데 이거는 말소된 기록이란 뜻이야.
    내가 말소기록을 요청할 경우에만 말소기록()을 사용하고 나머지 경우는 사용하지 마. 
    metadata가 다르면 다른 문서라는 의미야. 나는 메타데이터가 같은 문서에 대해 정보를 얻고 싶어
    <context>
    {context}
    </context>

    Question: {question}""")

retriever = vector_index.as_retriever(
    search_type="similarity", 
    search_kwargs={
        "k": 5, 
    } 
)

# open_llm = ChatOpenAI(
#     temperature=0.5,  # 창의성 (0.0 ~ 2.0)
#     max_tokens=2048,  # 최대 토큰수
#     model_name="gpt-3.5-turbo",  # 모델명
# )

repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
# repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, 
    max_new_tokens=2048,  
    temperature=0.1, 
    callbacks=[StreamingStdOutCallbackHandler()], 
    streaming=True,  
)

conv_chain = ConversationalRetrievalChain.from_llm(
    llm, 
    retriever=retriever,
    # chain_type="stuff", 
    combine_docs_chain_kwargs={"prompt": prompt},
    memory=memory
)

query_list = ["마크앤컴퍼니의 상호, 상호의 영어명, 본점, 액면가, 발행한 주식의 총수, 발행할 주식의 총수, 회사성립연월일, 등기번호, 등록번호에 대해 표형식으로 정리해줘"]
for query in query_list:
    ## open ai
    # result_open = conv_chain.invoke({"question": query, "chat_history": chat_history})
    # print(result_open)
    # chat_history.append((query, result_open["answer"]))
    
    ##  hugging face
    result = conv_chain.invoke({"question": query, "chat_history": chat_history})
    print()
    chat_history.append((query, result["answer"]))


Answer: | 항목 | 내용 |
| --- | --- |
| 상호 | 마크앤컴퍼니 |
| 영어명 | Mark & Company Inc. |
| 본점 | 서울특별시 서초구 강남대로 311, 705호(서초동, 드림플러스 강남) |
| 액면가 | 금 500 원 (2019.06.10 변경) |
| 발행한 주식의 총수 | 240,000 주 (2020.04.30 변경) |
| 발행할 주식의 총수 | 1,000,000 주 |
| 회사성립연월일 | 2019 년 03 월 11 일 |
| 등기번호 | 703850 |
| 등록번호 | 110111-7038501 |</s>


# Test

In [ ]:
query = "마크앤컴퍼니의 액면가는?"
result = conv_chain.invoke({"question": query, "chat_history": chat_history})

 What is the par value of Mark & Company Inc.?</s>
Answer: 마크앤컴퍼니(Mark & Company Inc.)의 액면가액은 500원입니다.</s>{'question': '마크앤컴퍼니의 액면가는?', 'chat_history': [HumanMessage(content='마크앤컴퍼니의 상호, 상호의 영어명, 본점, 액면가, 발행한 주식의 총수, 발행할 주식의 총수, 회사성립연월일, 등기번호, 등록번호에 대해 표형식으로 정리해줘'), AIMessage(content='\nAnswer: | 항목 | 내용 |\n| --- | --- |\n| 상호 | 마크앤컴퍼니 |\n| 영어명 | Mark & Company Inc. |\n| 본점 | 서울특별시 서초구 강남대로 311, 705호(서초동, 드림플러스 강남) |\n| 액면가 | 금 500 원 (2019.06.10 변경) |\n| 발행한 주식의 총수 | 240,000 주 (2020.04.30 변경) |\n| 발행할 주식의 총수 | 1,000,000 주 |\n| 회사성립연월일 | 2019 년 03 월 11 일 |\n| 등기번호 | 703850 |\n| 등록번호 | 110111-7038501 |</s>'), HumanMessage(content='마크앤컴퍼니의 액면가는?'), AIMessage(content='\nAnswer: 마크앤컴퍼니(Mark & Company Inc.)의 액면가액은 500원입니다.</s>')], 'answer': '\nAnswer: 마크앤컴퍼니(Mark & Company Inc.)의 액면가액은 500원입니다.</s>'}


In [ ]:
chat_history

[('마크앤컴퍼니의 상호, 상호의 영어명, 본점, 액면가, 발행한 주식의 총수, 발행할 주식의 총수, 회사성립연월일, 등기번호, 등록번호에 대해 표형식으로 정리해줘',
  '\nAnswer: | 항목 | 내용 |\n| --- | --- |\n| 상호 | 마크앤컴퍼니 |\n| 영어명 | Mark & Company Inc. |\n| 본점 | 서울특별시 서초구 강남대로 311, 705호(서초동, 드림플러스 강남) |\n| 액면가 | 금 500 원 (2019.06.10 변경) |\n| 발행한 주식의 총수 | 240,000 주 (2020.04.30 변경) |\n| 발행할 주식의 총수 | 1,000,000 주 |\n| 회사성립연월일 | 2019 년 03 월 11 일 |\n| 등기번호 | 703850 |\n| 등록번호 | 110111-7038501 |</s>')]